# 02_IBM-HHRR_ATTRITION_Parametro_v3


## Previous Tasks

### Import Libraries

In [1]:
# Generic Libraries
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  RobustScaler
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML  

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import  f1_score, fbeta_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [2]:
# Define general path:
path_general = r'C:\HHRR'
path_total = os.path.join(path_general,'01_total_models') 

In [3]:
# Model Libraries.

# Cross validation
from sklearn.model_selection import cross_val_score 

#------------- / Regresion Logistica /--------------
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

#--------------/ XGBoost /--------------------------
from xgboost import XGBClassifier
import xgboost as xgb

#-------------/ AdaBoost /--------------------------
from sklearn.ensemble import AdaBoostClassifier

#-------------/ CatBoost /--------------------------
from catboost import CatBoostClassifier

#------------/ Decission Tree /----------------------
from sklearn.tree import DecisionTreeClassifier

#------------/ Random Forest /-----------------------
from sklearn.ensemble import RandomForestClassifier

#-----------/   MLP /--------------------------------
from sklearn.neural_network import MLPClassifier

#------------/ KNN /----------------------------------
from sklearn.neighbors import KNeighborsClassifier

#------------/ Naive - Bayes /-------------------------
from sklearn.naive_bayes import GaussianNB


### Load Dataset

In [4]:
# Load dataset
df = pd.read_csv(r'C:\Users\TrendingPC\Desktop\PROYECTOS\HHRR-IBM_ATTRITION\HR-Employee-ML.csv', index_col=0)
df

,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,Education_Associate Degree,Education_Bachelor’s Degree,Education_Doctorate,Education_High School,...,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,annual_salary,economic_impact
0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,11,0,8,0,6,4,0,5,71916,14670.864
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,23,1,10,3,10,7,1,7,61560,12558.240
2,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,15,0,7,3,0,0,0,0,25080,4037.880
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,11,0,8,3,8,7,3,0,34908,6876.876
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,12,1,6,3,2,2,2,2,41616,8198.352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,17,1,17,3,5,2,0,3,30852,6077.844
1466,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,15,1,9,5,7,7,1,7,119892,25177.320
1467,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,20,1,6,0,6,2,0,3,73704,15035.616
1468,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,14,0,17,3,9,6,0,8,64680,13194.720


In [5]:
X = df.drop(columns='Attrition')
y = df['Attrition']

In [6]:
# Separation of the dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42,stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1029, 78), (441, 78), (1029,), (441,))

In [7]:
# Check dataset composition

print(" Fraudulent Count for Full data :  ",np.sum(y))
print(" Fraudulent Count for Train data : ",np.sum(y_train))
print(" Fraudulent Count for Test data :   ",np.sum(y_test))

 Fraudulent Count for Full data :   237
 Fraudulent Count for Train data :  166
 Fraudulent Count for Test data :    71


In [8]:
# Save the testing set for evaluation
X_test_saved = X_test.copy()
y_test_saved = y_test.copy()
print("Saved X_test & y_test")

Saved X_test & y_test


## 1.- Transformaciones de datos.

## Dataset Original

### Smote

In [9]:
# Import of specific libraries
from collections import Counter
from imblearn.over_sampling import SMOTE

# Initial situation
print('Original dataset shape %s' % Counter(y_train))

# Calculate OverSampling model
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print('Resampled dataset shape %s' % Counter(y_train_smote))

Original dataset shape Counter({0: 863, 1: 166})
Resampled dataset shape Counter({0: 863, 1: 863})


### Adasyn

In [10]:
# Import of specific libraries
from imblearn.over_sampling import ADASYN

# Initial situation
print('Original dataset shape %s' % Counter(y_train))

# Calculate OverSampling model
adasyn = ADASYN(random_state=42)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)

print('Resampled dataset shape %s' % Counter(y_train_adasyn))

Original dataset shape Counter({0: 863, 1: 166})
Resampled dataset shape Counter({0: 863, 1: 800})


## Power Transformation

### Original

In [11]:
# - Apply : preprocessing.PowerTransformer(copy=False) to fit & transform the train & test data

from sklearn import metrics 
from sklearn import preprocessing


from sklearn.preprocessing import PowerTransformer

pt= preprocessing.PowerTransformer(method='yeo-johnson', copy=True)  # creates an instance of the PowerTransformer class.
pt.fit(X_train)

X_train_pt = pt.transform(X_train)
X_test_pt = pt.transform(X_test)

y_train_pt = y_train
y_test_pt = y_test

### Smote

In [12]:
# Import of specific libraries
from collections import Counter
from imblearn.over_sampling import SMOTE

# Initial situation
print('Original dataset shape %s' % Counter(y_train_pt))

# Calculate OverSampling model
smote = SMOTE(random_state=42)
X_train_smote_pt, y_train_smote_pt = smote.fit_resample(X_train_pt, y_train_pt)

print('Resampled dataset shape %s' % Counter(y_train_smote_pt))

Original dataset shape Counter({0: 863, 1: 166})
Resampled dataset shape Counter({0: 863, 1: 863})


### Adasyn

In [13]:
# Import of specific libraries
from imblearn.over_sampling import ADASYN

# Initial situation
print('Original dataset shape %s' % Counter(y_train))

# Calculate OverSampling model
adasyn = ADASYN(random_state=42)
X_train_adasyn_pt, y_train_adasyn_pt = adasyn.fit_resample(X_train_pt, y_train_pt)

print('Resampled dataset shape %s' % Counter(y_train_adasyn_pt))

Original dataset shape Counter({0: 863, 1: 166})
Resampled dataset shape Counter({0: 863, 1: 839})


### Load Model: Libraries and Functions.

In [14]:
# LOAD OF MODELS.
# perfom cross validation on the X_train & y_train 
from sklearn.model_selection import StratifiedKFold

# Initialize StratifiedKFold cross-validator
# perform cross validation
skf = StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
#  Shuffle is False because we need a constant best model when we use GridSearchCV


In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict

### Create dataset_list

In [16]:
# Original distribution
OR_origin = ['OR origin',X_train, y_train, X_test, y_test]
OR_smote =['OR smote',X_train_smote, y_train_smote, X_test, y_test]
OR_adasyn = ['OR adasyn', X_train_adasyn, y_train_adasyn, X_test, y_test]

# Power Transformation
PT_origin = ['PT origin',X_train_pt, y_train_pt, X_test_pt, y_test_pt]
PT_smote = ['PT smote',X_train_smote_pt, y_train_smote_pt, X_test_pt, y_test_pt ]
PT_adasyn = ['PT adasyn', X_train_adasyn_pt, y_train_adasyn_pt, X_test_pt, y_test_pt]

### Create models

In [17]:
model_list = ['regression_logistic', 'adaboost', 'xgboost', 'catboost', 'decision_tree', 'random_forest', 'mlp', 'knn']
parameters = [
    [0.1, 0.5, 1, 1.5, 2, 2.5, 3],      # For 'regression_logistic'
    [5, 7, 9],                          # For 'adaboost'
    [0.001, 0.01, 0.1, 0.5, 1, 3],      # For 'xgboost'
    [100, 200, 300, 400, 500, 600],      # For 'catboost'
    [1, 2, 3, 4, 5],                    # For 'decision_tree'
    [100, 200, 400, 600, 800, 1000, 1200],  # For 'random_forest'
    [(50,), (100,), (120,), (150,)],     # For 'mlp'
    [3, 5, 7]  #knn
    ]

In [18]:
distributions =[OR_origin, OR_smote, OR_adasyn, PT_origin, PT_smote, PT_adasyn]


distributions =[OR_origin, OR_smote, OR_adasyn, PT_origin, PT_smote, PT_adasyn]
complete_model = zip(model_list, parameters)
complete_model_list = list(complete_model )
complete_model_list

In [19]:
pd.set_option('display.max_colwidth', None)

In [20]:
import os
import pandas as pd
from itertools import product
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score, recall_score, 
    f1_score, fbeta_score, confusion_matrix
)
from IPython.display import display, HTML

def gen_models_with_three_params(complete_model_list, distributions, save_directory_complete_model=None):
    """
    Trains models with three levels of parameters and saves the results.

    Parameters:
    -----------
    complete_model_list : list
        List of tuples where each tuple contains:
        - model_name (str): Name of the model.
        - param_grid (dict): Dictionary with lists of values for three hyperparameters.

    distributions : list
        List of datasets where each entry contains:
        - Name of the distribution.
        - X_train, y_train, X_val, y_val.

    save_directory_complete_model : str, optional
        Directory to save the results. Defaults to a folder named 'total' in the current working directory.

    Returns:
    --------
    DataFrame
        Combined results of all models for further processing.
    """

    # Create directory if not provided
    if save_directory_complete_model is None:
        save_directory_complete_model = os.path.join(os.getcwd(), 'total')
    os.makedirs(save_directory_complete_model, exist_ok=True)

    all_results = []  # To store results from all models

    # Iterate over models
    for model_name, param_grid in complete_model_list:
        resultados_totales = []

        # Generate combinations of parameters
        keys, values = zip(*param_grid.items())
        param_combinations = [dict(zip(keys, v)) for v in product(*values)]

        # Iterate over distributions
        for distribution in distributions:
            try:
                # Unpack distribution
                name = distribution[0]
                X_train, y_train, X_val, y_val = distribution[1:]

                # Skip invalid data
                if X_train is None or y_train is None or X_val is None or y_val is None:
                    print(f"    Skipping due to missing data in {name}")
                    continue

                # Iterate over parameter combinations
                for params in param_combinations:
                    # Initialize model with parameters
                    if model_name == 'regression_logistic':
                        model_instance = LogisticRegression(
                            C=params['C'],
                            penalty=params['penalty'],
                            max_iter=params['max_iter']
                        )
                        parameter_description = f"C={params['C']}, penalty={params['penalty']}, max_iter={params['max_iter']}"
                    elif model_name == 'random_forest':
                        model_instance = RandomForestClassifier(
                            n_estimators=params['n_estimators'],
                            criterion=params['criterion'],
                            max_depth=params['max_depth']
                        )
                        parameter_description = f"n_estimators={params['n_estimators']}, criterion={params['criterion']}, max_depth={params['max_depth']}"
                    elif model_name == 'xgboost':
                        model_instance = XGBClassifier(
                            learning_rate=params['learning_rate'],
                            booster=params['booster'],
                            n_estimators=params['n_estimators']
                        )
                        parameter_description = f"learning_rate={params['learning_rate']}, booster={params['booster']}, n_estimators={params['n_estimators']}"
                    elif model_name == 'decision_tree':
                        model_instance = DecisionTreeClassifier(
                            criterion=params['criterion'],
                            max_depth=params['max_depth'],
                        )
                        parameter_description = f"criterion={params['criterion']}, max_depth={params['max_depth']}"
                    elif model_name == 'adaboost':
                        model_instance = AdaBoostClassifier(
                            n_estimators=params['n_estimators'],
                            learning_rate=params['learning_rate'],
                            algorithm=params['algorithm']
                        )
                        parameter_description = f"n_estimators={params['n_estimators']}, learning_rate={params['learning_rate']}, algorithm={params['algorithm']}"
                    elif model_name == 'catboost':
                        model_instance = CatBoostClassifier(
                            learning_rate=params['learning_rate'],
                            depth=params['depth'],
                            iterations=params['iterations'],
                            verbose=0  # Suppress verbose output
                        )
                        parameter_description = f"learning_rate={params['learning_rate']}, depth={params['depth']}, iterations={params['iterations']}"
                    else:
                        print(f"    Invalid model name: {model_name}")
                        continue

                    # Train model
                    model_instance.fit(X_train, y_train)

                    # Get predictions
                    y_pred = model_instance.predict(X_val)

                    # Calculate metrics
                    roc_auc = roc_auc_score(y_val, y_pred)
                    accuracy = accuracy_score(y_val, y_pred)
                    precision = precision_score(y_val, y_pred)
                    recall = recall_score(y_val, y_pred)
                    f1 = f1_score(y_val, y_pred)
                    f2 = fbeta_score(y_val, y_pred, beta=2)
                    confusion = confusion_matrix(y_val, y_pred)

                    # Save results in DataFrame
                    results_df = pd.DataFrame({
                        'Model': [model_name],
                        'Description': [name],
                        'Parameter_Description': [parameter_description],
                        'ROC-AUC': [roc_auc],
                        'Accuracy': [accuracy],
                        'Precision': [precision],
                        'Recall': [recall],
                        'F1 Score': [f1],
                        'F2 Score': [f2],
                        'Confusion Matrix': [confusion.tolist()],  # Convert to list for compatibility
                    })

                    resultados_totales.append(results_df)

            except Exception as e:
                print(f"  Error processing distribution {name}: {str(e)}")
                continue

        # Save results for this model
        if resultados_totales:
            df_resultados_final = pd.concat(resultados_totales, ignore_index=True)
            save_path = os.path.join(save_directory_complete_model, f"{model_name}_total.csv")
            df_resultados_final.to_csv(save_path, index=False)
            display(HTML(f"<h2 style='text-align: center;font-size:60px;'> Modelo: {model_name}</h2>"))
            display(df_resultados_final)
            print(f"\n\n\nResults for {model_name} saved to {save_path}")
            all_results.append(df_resultados_final)
        else:
            print(f"No results generated for {model_name}")

    # Combine all results into a single DataFrame
    if all_results:
        combined_results = pd.concat(all_results, ignore_index=True)
        return combined_results
    else:
        print("No results generated for any model")
        return pd.DataFrame()


In [21]:
complete_model_list = [
    ('regression_logistic', {'C': [2, 2.5, 3 ], 'penalty': ['none'], 'max_iter': [ 125,150, 200]})]

In [22]:
reg_log_opt = gen_models_with_three_params(complete_model_list, distributions)

,Model,Description,Parameter_Description,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,regression_logistic,OR origin,"C=2, penalty=none, max_iter=125",0.505691,0.839002,0.500000,0.014085,0.027397,0.017483,"[[369, 1], [70, 1]]"
1,regression_logistic,OR origin,"C=2, penalty=none, max_iter=150",0.505691,0.839002,0.500000,0.014085,0.027397,0.017483,"[[369, 1], [70, 1]]"
2,regression_logistic,OR origin,"C=2, penalty=none, max_iter=200",0.514085,0.843537,1.000000,0.028169,0.054795,0.034965,"[[370, 0], [69, 2]]"
3,regression_logistic,OR origin,"C=2.5, penalty=none, max_iter=125",0.505691,0.839002,0.500000,0.014085,0.027397,0.017483,"[[369, 1], [70, 1]]"
4,regression_logistic,OR origin,"C=2.5, penalty=none, max_iter=150",0.505691,0.839002,0.500000,0.014085,0.027397,0.017483,"[[369, 1], [70, 1]]"
5,regression_logistic,OR origin,"C=2.5, penalty=none, max_iter=200",0.514085,0.843537,1.000000,0.028169,0.054795,0.034965,"[[370, 0], [69, 2]]"
6,regression_logistic,OR origin,"C=3, penalty=none, max_iter=125",0.505691,0.839002,0.500000,0.014085,0.027397,0.017483,"[[369, 1], [70, 1]]"
7,regression_logistic,OR origin,"C=3, penalty=none, max_iter=150",0.505691,0.839002,0.500000,0.014085,0.027397,0.017483,"[[369, 1], [70, 1]]"
8,regression_logistic,OR origin,"C=3, penalty=none, max_iter=200",0.514085,0.843537,1.000000,0.028169,0.054795,0.034965,"[[370, 0], [69, 2]]"
9,regression_logistic,OR smote,"C=2, penalty=none, max_iter=125",0.615188,0.659864,0.248408,0.549296,0.342105,0.442177,"[[252, 118], [32, 39]]"





Results for regression_logistic saved to C:\Users\TrendingPC\Desktop\PROYECTOS\HHRR-IBM_ATTRITION\total\regression_logistic_total.csv


In [23]:
reg_log_opt = reg_log_opt.sort_values(by='ROC-AUC', ascending=False)
reg_log_opt

,Model,Description,Parameter_Description,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
53,regression_logistic,PT adasyn,"C=3, penalty=none, max_iter=200",0.745851,0.802721,0.427273,0.661972,0.519337,0.596447,"[[307, 63], [24, 47]]"
52,regression_logistic,PT adasyn,"C=3, penalty=none, max_iter=150",0.745851,0.802721,0.427273,0.661972,0.519337,0.596447,"[[307, 63], [24, 47]]"
51,regression_logistic,PT adasyn,"C=3, penalty=none, max_iter=125",0.745851,0.802721,0.427273,0.661972,0.519337,0.596447,"[[307, 63], [24, 47]]"
50,regression_logistic,PT adasyn,"C=2.5, penalty=none, max_iter=200",0.745851,0.802721,0.427273,0.661972,0.519337,0.596447,"[[307, 63], [24, 47]]"
49,regression_logistic,PT adasyn,"C=2.5, penalty=none, max_iter=150",0.745851,0.802721,0.427273,0.661972,0.519337,0.596447,"[[307, 63], [24, 47]]"
48,regression_logistic,PT adasyn,"C=2.5, penalty=none, max_iter=125",0.745851,0.802721,0.427273,0.661972,0.519337,0.596447,"[[307, 63], [24, 47]]"
47,regression_logistic,PT adasyn,"C=2, penalty=none, max_iter=200",0.745851,0.802721,0.427273,0.661972,0.519337,0.596447,"[[307, 63], [24, 47]]"
46,regression_logistic,PT adasyn,"C=2, penalty=none, max_iter=150",0.745851,0.802721,0.427273,0.661972,0.519337,0.596447,"[[307, 63], [24, 47]]"
45,regression_logistic,PT adasyn,"C=2, penalty=none, max_iter=125",0.745851,0.802721,0.427273,0.661972,0.519337,0.596447,"[[307, 63], [24, 47]]"
40,regression_logistic,PT smote,"C=2.5, penalty=none, max_iter=150",0.733118,0.800454,0.420561,0.633803,0.505618,0.575448,"[[308, 62], [26, 45]]"


In [74]:
reg_log_best = reg_log_opt.iloc[0]
reg_log_best

Model                                regression_logistic
Description                                    PT adasyn
Parameter_Description    C=3, penalty=none, max_iter=200
ROC-AUC                                         0.745851
Accuracy                                        0.802721
Precision                                       0.427273
Recall                                          0.661972
F1 Score                                        0.519337
F2 Score                                        0.596447
Confusion Matrix                   [[307, 63], [24, 47]]
Name: 53, dtype: object

In [25]:
complete_model_list = [('decision_tree', {'criterion': ['gini'], 'max_depth': [3, 5], 'max_features': [5, 7,9]})]

In [26]:
dec_tree_opt = gen_models_with_three_params(complete_model_list, distributions)

,Model,Description,Parameter_Description,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,decision_tree,OR origin,"criterion=gini, max_depth=3",0.591054,0.839002,0.500000,0.225352,0.310680,0.253165,"[[354, 16], [55, 16]]"
1,decision_tree,OR origin,"criterion=gini, max_depth=3",0.592406,0.841270,0.516129,0.225352,0.313725,0.253968,"[[355, 15], [55, 16]]"
2,decision_tree,OR origin,"criterion=gini, max_depth=3",0.592406,0.841270,0.516129,0.225352,0.313725,0.253968,"[[355, 15], [55, 16]]"
3,decision_tree,OR origin,"criterion=gini, max_depth=5",0.565093,0.804989,0.333333,0.211268,0.258621,0.227964,"[[340, 30], [56, 15]]"
4,decision_tree,OR origin,"criterion=gini, max_depth=5",0.560754,0.807256,0.333333,0.197183,0.247788,0.214724,"[[342, 28], [57, 14]]"
5,decision_tree,OR origin,"criterion=gini, max_depth=5",0.560754,0.807256,0.333333,0.197183,0.247788,0.214724,"[[342, 28], [57, 14]]"
6,decision_tree,OR smote,"criterion=gini, max_depth=3",0.627903,0.843537,0.523810,0.309859,0.389381,0.337423,"[[350, 20], [49, 22]]"
7,decision_tree,OR smote,"criterion=gini, max_depth=3",0.627903,0.843537,0.523810,0.309859,0.389381,0.337423,"[[350, 20], [49, 22]]"
8,decision_tree,OR smote,"criterion=gini, max_depth=3",0.627903,0.843537,0.523810,0.309859,0.389381,0.337423,"[[350, 20], [49, 22]]"
9,decision_tree,OR smote,"criterion=gini, max_depth=5",0.613038,0.818594,0.415094,0.309859,0.354839,0.326409,"[[339, 31], [49, 22]]"





Results for decision_tree saved to C:\Users\TrendingPC\Desktop\PROYECTOS\HHRR-IBM_ATTRITION\total\decision_tree_total.csv


In [27]:
dec_tree_opt = dec_tree_opt.sort_values(by='ROC-AUC', ascending=False)

In [28]:
dec_tree_best = dec_tree_opt.iloc[0]

In [29]:
dec_tree_best

Model                                  decision_tree
Description                                PT adasyn
Parameter_Description    criterion=gini, max_depth=3
ROC-AUC                                     0.706585
Accuracy                                     0.78458
Precision                                   0.388889
Recall                                      0.591549
F1 Score                                    0.469274
F2 Score                                    0.535714
Confusion Matrix               [[304, 66], [29, 42]]
Name: 32, dtype: object

In [30]:
complete_model_list = [('random_forest', {'n_estimators':[200, 250],'criterion':['entropy'], 'max_depth': [3, 5]}) ]

In [31]:
random_forest_opt = gen_models_with_three_params(complete_model_list, distributions)

,Model,Description,Parameter_Description,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,random_forest,OR origin,"n_estimators=200, criterion=entropy, max_depth=3",0.508679,0.834467,0.333333,0.028169,0.051948,0.034483,"[[366, 4], [69, 2]]"
1,random_forest,OR origin,"n_estimators=200, criterion=entropy, max_depth=5",0.531443,0.834467,0.428571,0.084507,0.141176,0.100671,"[[362, 8], [65, 6]]"
2,random_forest,OR origin,"n_estimators=250, criterion=entropy, max_depth=3",0.511667,0.829932,0.300000,0.042254,0.074074,0.051020,"[[363, 7], [68, 3]]"
3,random_forest,OR origin,"n_estimators=250, criterion=entropy, max_depth=5",0.524400,0.832200,0.384615,0.070423,0.119048,0.084175,"[[362, 8], [66, 5]]"
4,random_forest,OR smote,"n_estimators=200, criterion=entropy, max_depth=3",0.660697,0.841270,0.509091,0.394366,0.444444,0.412979,"[[343, 27], [43, 28]]"
5,random_forest,OR smote,"n_estimators=200, criterion=entropy, max_depth=5",0.653083,0.857143,0.595238,0.352113,0.442478,0.383436,"[[353, 17], [46, 25]]"
6,random_forest,OR smote,"n_estimators=250, criterion=entropy, max_depth=3",0.646898,0.827664,0.457627,0.380282,0.415385,0.393586,"[[338, 32], [44, 27]]"
7,random_forest,OR smote,"n_estimators=250, criterion=entropy, max_depth=5",0.633308,0.852608,0.578947,0.309859,0.403670,0.341615,"[[354, 16], [49, 22]]"
8,random_forest,OR adasyn,"n_estimators=200, criterion=entropy, max_depth=3",0.641987,0.848073,0.545455,0.338028,0.417391,0.365854,"[[350, 20], [47, 24]]"
9,random_forest,OR adasyn,"n_estimators=200, criterion=entropy, max_depth=5",0.610544,0.852608,0.600000,0.253521,0.356436,0.286624,"[[358, 12], [53, 18]]"





Results for random_forest saved to C:\Users\TrendingPC\Desktop\PROYECTOS\HHRR-IBM_ATTRITION\total\random_forest_total.csv


In [32]:
random_forest_opt = random_forest_opt.sort_values(by='ROC-AUC', ascending=False)
random_forest_opt

,Model,Description,Parameter_Description,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
18,random_forest,PT smote,"n_estimators=250, criterion=entropy, max_depth=3",0.709783,0.818594,0.448276,0.549296,0.493671,0.525606,"[[322, 48], [32, 39]]"
22,random_forest,PT adasyn,"n_estimators=250, criterion=entropy, max_depth=3",0.702741,0.816327,0.441860,0.535211,0.484076,0.513514,"[[322, 48], [33, 38]]"
19,random_forest,PT smote,"n_estimators=250, criterion=entropy, max_depth=5",0.691854,0.845805,0.523810,0.464789,0.492537,0.475504,"[[340, 30], [38, 33]]"
20,random_forest,PT adasyn,"n_estimators=200, criterion=entropy, max_depth=3",0.685668,0.816327,0.437500,0.492958,0.463576,0.480769,"[[325, 45], [36, 35]]"
16,random_forest,PT smote,"n_estimators=200, criterion=entropy, max_depth=3",0.683251,0.802721,0.409091,0.507042,0.452830,0.483871,"[[318, 52], [35, 36]]"
17,random_forest,PT smote,"n_estimators=200, criterion=entropy, max_depth=5",0.674781,0.845805,0.526316,0.422535,0.468750,0.439883,"[[343, 27], [41, 30]]"
4,random_forest,OR smote,"n_estimators=200, criterion=entropy, max_depth=3",0.660697,0.841270,0.509091,0.394366,0.444444,0.412979,"[[343, 27], [43, 28]]"
5,random_forest,OR smote,"n_estimators=200, criterion=entropy, max_depth=5",0.653083,0.857143,0.595238,0.352113,0.442478,0.383436,"[[353, 17], [46, 25]]"
21,random_forest,PT adasyn,"n_estimators=200, criterion=entropy, max_depth=5",0.649600,0.832200,0.473684,0.380282,0.421875,0.395894,"[[340, 30], [44, 27]]"
6,random_forest,OR smote,"n_estimators=250, criterion=entropy, max_depth=3",0.646898,0.827664,0.457627,0.380282,0.415385,0.393586,"[[338, 32], [44, 27]]"


In [75]:
random_forest_best = random_forest_opt.iloc[0]
random_forest_best

Model                                                       random_forest
Description                                                      PT smote
Parameter_Description    n_estimators=250, criterion=entropy, max_depth=3
ROC-AUC                                                          0.709783
Accuracy                                                         0.818594
Precision                                                        0.448276
Recall                                                           0.549296
F1 Score                                                         0.493671
F2 Score                                                         0.525606
Confusion Matrix                                    [[322, 48], [32, 39]]
Name: 18, dtype: object

In [70]:
complete_model_list = [
    ('xgboost', {'learning_rate': [0.001, 0.05, 0.1], 'booster': ['gblinear'], 'n_estimators': [50, 100, 200]})]

In [71]:
xgboost_opt = gen_models_with_three_params(complete_model_list, distributions, path_total)

,Model,Description,Parameter_Description,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,xgboost,OR origin,"learning_rate=0.001, booster=gblinear, n_estimators=50",0.500000,0.839002,0.000000,0.000000,0.000000,0.000000,"[[370, 0], [71, 0]]"
1,xgboost,OR origin,"learning_rate=0.001, booster=gblinear, n_estimators=100",0.500000,0.839002,0.000000,0.000000,0.000000,0.000000,"[[370, 0], [71, 0]]"
2,xgboost,OR origin,"learning_rate=0.001, booster=gblinear, n_estimators=200",0.500000,0.839002,0.000000,0.000000,0.000000,0.000000,"[[370, 0], [71, 0]]"
3,xgboost,OR origin,"learning_rate=0.05, booster=gblinear, n_estimators=50",0.635725,0.866213,0.700000,0.295775,0.415842,0.334395,"[[361, 9], [50, 21]]"
4,xgboost,OR origin,"learning_rate=0.05, booster=gblinear, n_estimators=100",0.685306,0.873016,0.674419,0.408451,0.508772,0.443425,"[[356, 14], [42, 29]]"
5,xgboost,OR origin,"learning_rate=0.05, booster=gblinear, n_estimators=200",0.688295,0.868481,0.638298,0.422535,0.508475,0.453172,"[[353, 17], [41, 30]]"
6,xgboost,OR origin,"learning_rate=0.1, booster=gblinear, n_estimators=50",0.692349,0.875283,0.681818,0.422535,0.521739,0.457317,"[[356, 14], [41, 30]]"
7,xgboost,OR origin,"learning_rate=0.1, booster=gblinear, n_estimators=100",0.688295,0.868481,0.638298,0.422535,0.508475,0.453172,"[[353, 17], [41, 30]]"
8,xgboost,OR origin,"learning_rate=0.1, booster=gblinear, n_estimators=200",0.701028,0.870748,0.640000,0.450704,0.528926,0.479042,"[[352, 18], [39, 32]]"
9,xgboost,OR smote,"learning_rate=0.001, booster=gblinear, n_estimators=50",0.720099,0.807256,0.428571,0.591549,0.497041,0.549738,"[[314, 56], [29, 42]]"





Results for xgboost saved to C:\HHRR\01_total_models\xgboost_total.csv


In [72]:
xgboost_opt = xgboost_opt.sort_values(by='F2 Score', ascending=False)
xgboost_opt

,Model,Description,Parameter_Description,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
48,xgboost,PT adasyn,"learning_rate=0.05, booster=gblinear, n_estimators=50",0.766483,0.780045,0.401515,0.746479,0.522167,0.637019,"[[291, 79], [18, 53]]"
39,xgboost,PT smote,"learning_rate=0.05, booster=gblinear, n_estimators=50",0.755101,0.780045,0.398438,0.718310,0.512563,0.618932,"[[293, 77], [20, 51]]"
12,xgboost,OR smote,"learning_rate=0.05, booster=gblinear, n_estimators=50",0.750190,0.800454,0.424779,0.676056,0.521739,0.604534,"[[305, 65], [23, 48]]"
51,xgboost,PT adasyn,"learning_rate=0.1, booster=gblinear, n_estimators=50",0.746422,0.784580,0.401639,0.690141,0.507772,0.603448,"[[297, 73], [22, 49]]"
49,xgboost,PT adasyn,"learning_rate=0.05, booster=gblinear, n_estimators=100",0.746422,0.784580,0.401639,0.690141,0.507772,0.603448,"[[297, 73], [22, 49]]"
40,xgboost,PT smote,"learning_rate=0.05, booster=gblinear, n_estimators=100",0.746136,0.793651,0.413793,0.676056,0.513369,0.600000,"[[302, 68], [23, 48]]"
43,xgboost,PT smote,"learning_rate=0.1, booster=gblinear, n_estimators=100",0.744499,0.800454,0.423423,0.661972,0.516484,0.594937,"[[306, 64], [24, 47]]"
41,xgboost,PT smote,"learning_rate=0.05, booster=gblinear, n_estimators=200",0.744499,0.800454,0.423423,0.661972,0.516484,0.594937,"[[306, 64], [24, 47]]"
44,xgboost,PT smote,"learning_rate=0.1, booster=gblinear, n_estimators=200",0.745565,0.811791,0.442308,0.647887,0.525714,0.592784,"[[312, 58], [25, 46]]"
42,xgboost,PT smote,"learning_rate=0.1, booster=gblinear, n_estimators=50",0.741797,0.795918,0.415929,0.661972,0.510870,0.591940,"[[304, 66], [24, 47]]"


In [73]:
xgboost_best = xgboost_opt.iloc[0]
xgboost_best

Model                                                                  xgboost
Description                                                          PT adasyn
Parameter_Description    learning_rate=0.05, booster=gblinear, n_estimators=50
ROC-AUC                                                               0.766483
Accuracy                                                              0.780045
Precision                                                             0.401515
Recall                                                                0.746479
F1 Score                                                              0.522167
F2 Score                                                              0.637019
Confusion Matrix                                         [[291, 79], [18, 53]]
Name: 48, dtype: object

In [78]:
complete_model_list = [
    ('adaboost', {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1], 'algorithm': ['SAMME', 'SAMME.R']})]

In [79]:
adaboost_opt = gen_models_with_three_params(complete_model_list, distributions, path_total)

,Model,Description,Parameter_Description,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,adaboost,OR origin,"n_estimators=50, learning_rate=0.01, algorithm=SAMME",0.500000,0.839002,0.000000,0.000000,0.000000,0.000000,"[[370, 0], [71, 0]]"
1,adaboost,OR origin,"n_estimators=50, learning_rate=0.01, algorithm=SAMME.R",0.500000,0.839002,0.000000,0.000000,0.000000,0.000000,"[[370, 0], [71, 0]]"
2,adaboost,OR origin,"n_estimators=50, learning_rate=0.1, algorithm=SAMME",0.514085,0.843537,1.000000,0.028169,0.054795,0.034965,"[[370, 0], [69, 2]]"
3,adaboost,OR origin,"n_estimators=50, learning_rate=0.1, algorithm=SAMME.R",0.518424,0.841270,0.600000,0.042254,0.078947,0.051903,"[[368, 2], [68, 3]]"
4,adaboost,OR origin,"n_estimators=50, learning_rate=1, algorithm=SAMME",0.606776,0.836735,0.487179,0.267606,0.345455,0.294118,"[[350, 20], [52, 19]]"
...,...,...,...,...,...,...,...,...,...,...
103,adaboost,PT adasyn,"n_estimators=200, learning_rate=0.01, algorithm=SAMME.R",0.680053,0.768707,0.357798,0.549296,0.433333,0.496183,"[[300, 70], [32, 39]]"
104,adaboost,PT adasyn,"n_estimators=200, learning_rate=0.1, algorithm=SAMME",0.697335,0.807256,0.422222,0.535211,0.472050,0.508021,"[[318, 52], [33, 38]]"
105,adaboost,PT adasyn,"n_estimators=200, learning_rate=0.1, algorithm=SAMME.R",0.682394,0.829932,0.471429,0.464789,0.468085,0.466102,"[[333, 37], [38, 33]]"
106,adaboost,PT adasyn,"n_estimators=200, learning_rate=1, algorithm=SAMME",0.672078,0.841270,0.508475,0.422535,0.461538,0.437318,"[[341, 29], [41, 30]]"





Results for adaboost saved to C:\HHRR\01_total_models\adaboost_total.csv


,Model,Description,Parameter_Description,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,adaboost,OR origin,"n_estimators=50, learning_rate=0.01, algorithm=SAMME",0.500000,0.839002,0.000000,0.000000,0.000000,0.000000,"[[370, 0], [71, 0]]"
1,adaboost,OR origin,"n_estimators=50, learning_rate=0.01, algorithm=SAMME.R",0.500000,0.839002,0.000000,0.000000,0.000000,0.000000,"[[370, 0], [71, 0]]"
2,adaboost,OR origin,"n_estimators=50, learning_rate=0.1, algorithm=SAMME",0.514085,0.843537,1.000000,0.028169,0.054795,0.034965,"[[370, 0], [69, 2]]"
3,adaboost,OR origin,"n_estimators=50, learning_rate=0.1, algorithm=SAMME.R",0.518424,0.841270,0.600000,0.042254,0.078947,0.051903,"[[368, 2], [68, 3]]"
4,adaboost,OR origin,"n_estimators=50, learning_rate=1, algorithm=SAMME",0.606776,0.836735,0.487179,0.267606,0.345455,0.294118,"[[350, 20], [52, 19]]"
...,...,...,...,...,...,...,...,...,...,...
103,adaboost,PT adasyn,"n_estimators=200, learning_rate=0.01, algorithm=SAMME.R",0.680053,0.768707,0.357798,0.549296,0.433333,0.496183,"[[300, 70], [32, 39]]"
104,adaboost,PT adasyn,"n_estimators=200, learning_rate=0.1, algorithm=SAMME",0.697335,0.807256,0.422222,0.535211,0.472050,0.508021,"[[318, 52], [33, 38]]"
105,adaboost,PT adasyn,"n_estimators=200, learning_rate=0.1, algorithm=SAMME.R",0.682394,0.829932,0.471429,0.464789,0.468085,0.466102,"[[333, 37], [38, 33]]"
106,adaboost,PT adasyn,"n_estimators=200, learning_rate=1, algorithm=SAMME",0.672078,0.841270,0.508475,0.422535,0.461538,0.437318,"[[341, 29], [41, 30]]"


In [81]:
adaboost_opt = adaboost_opt.sort_values(by='ROC-AUC', ascending=False)

In [82]:
adaboost_best = adaboost_opt.iloc[0]
adaboost_best

Model                                                                adaboost
Description                                                         PT adasyn
Parameter_Description    n_estimators=100, learning_rate=1, algorithm=SAMME.R
ROC-AUC                                                              0.715969
Accuracy                                                             0.848073
Precision                                                            0.528571
Recall                                                               0.521127
F1 Score                                                             0.524823
F2 Score                                                             0.522599
Confusion Matrix                                        [[337, 33], [34, 37]]
Name: 101, dtype: object

In [85]:
complete_model_list = [
    ('catboost', {'learning_rate': [0.01, 0.1, 0.3], 'iterations': [100, 500, 1000], 'depth': [3, 6, 10]})]

In [88]:
catboost_opt = gen_models_with_three_params(complete_model_list, distributions, path_total)

,Model,Description,Parameter_Description,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,catboost,OR origin,"learning_rate=0.01, depth=3, iterations=100",0.512733,0.841270,0.666667,0.028169,0.054054,0.034843,"[[369, 1], [69, 2]]"
1,catboost,OR origin,"learning_rate=0.01, depth=6, iterations=100",0.535497,0.841270,0.545455,0.084507,0.146341,0.101695,"[[365, 5], [65, 6]]"
2,catboost,OR origin,"learning_rate=0.01, depth=10, iterations=100",0.528455,0.839002,0.500000,0.070423,0.123457,0.085034,"[[365, 5], [66, 5]]"
3,catboost,OR origin,"learning_rate=0.01, depth=3, iterations=500",0.566654,0.845805,0.578947,0.154930,0.244444,0.181518,"[[362, 8], [60, 11]]"
4,catboost,OR origin,"learning_rate=0.01, depth=6, iterations=500",0.573696,0.848073,0.600000,0.169014,0.263736,0.197368,"[[362, 8], [59, 12]]"
...,...,...,...,...,...,...,...,...,...,...
157,catboost,PT adasyn,"learning_rate=0.3, depth=6, iterations=500",0.665531,0.868481,0.666667,0.366197,0.472727,0.402477,"[[357, 13], [45, 26]]"
158,catboost,PT adasyn,"learning_rate=0.3, depth=10, iterations=500",0.653864,0.877551,0.793103,0.323944,0.460000,0.367412,"[[364, 6], [48, 23]]"
159,catboost,PT adasyn,"learning_rate=0.3, depth=3, iterations=1000",0.645756,0.863946,0.657143,0.323944,0.433962,0.360502,"[[358, 12], [48, 23]]"
160,catboost,PT adasyn,"learning_rate=0.3, depth=6, iterations=1000",0.657137,0.863946,0.641026,0.352113,0.454545,0.386997,"[[356, 14], [46, 25]]"





Results for catboost saved to C:\HHRR\01_total_models\catboost_total.csv


In [89]:
catboost_opt = catboost_opt.sort_values(by='ROC-AUC', ascending=False)

In [91]:
catboost_best = catboost_opt.iloc[0]
catboost_best

Model                                                       catboost
Description                                                 PT smote
Parameter_Description    learning_rate=0.01, depth=3, iterations=100
ROC-AUC                                                     0.692425
Accuracy                                                    0.827664
Precision                                                   0.466667
Recall                                                      0.492958
F1 Score                                                    0.479452
F2 Score                                                    0.487465
Confusion Matrix                               [[330, 40], [36, 35]]
Name: 108, dtype: object